In [4]:
import pandas as pd
import json

In [5]:
conv_path = '/data/LPJ/ICML25/GraphCoder/graphgpt_dataset/less_acl25_480_eval/eval/conversations_30.json'
graph_path = '/data/LPJ/ICML25/GraphCoder/graphgpt_dataset/less_acl25_480_eval/eval/graph_30.jsonl'


In [6]:
with open(conv_path, 'r') as f:
    conv = json.load(f)

graph = pd.read_json(graph_path, lines=True)


In [9]:
conv[12]

{'conversations': [{'from': 'human',
   'value': "Given a submodules interconnection graph: \n<graph>\n, \nnodes: [{'id': 0, 'content': 'clk', 'type': 'input port'}, {'id': 1, 'content': 'reset', 'type': 'input port'}, {'id': 2, 'content': 'load', 'type': 'input port'}, {'id': 3, 'content': 'shift_dir', 'type': 'input port'}, {'id': 4, 'content': 'data_in', 'type': 'input port'}, {'id': 5, 'content': 'count_out', 'type': 'output port'}, {'id': 6, 'content': 'u_shift_logic', 'type': 'submodule'}, {'id': 7, 'content': 'u_shift_register', 'type': 'submodule'}], \nedge_attrs: [], \nconnectivity: [[3, 7, 4, 0, 2, 1, 6, 7], [6, 6, 7, 7, 7, 7, 7, 5]]\n, Design a Verilog module for a 4-bit circular shift register counter with a dynamic shift direction based on control inputs.\n\nModule name:\n    dynamic_shift_counter\n\nFunction:\nA 4-bit counter that shifts either left or right based on control signals. Additionally, the counter can load a specific value to start with, making it versatile fo

In [11]:
print(conv[12]['conversations'][1]['value'])

module dynamic_shift_counter (
    input wire clk,
    input wire reset,
    input wire load,
    input wire shift_dir,
    input wire [3:0] data_in,
    output wire [3:0] count_out
);

    wire [3:0] next_count;

    // Instantiate the shift logic
    shift_logic u_shift_logic (
        .shift_dir(shift_dir),
        .current_count(count_out),
        .next_count(next_count)
    );

    // Instantiate the register to hold the count value
    shift_register u_shift_register (
        .clk(clk),
        .reset(reset),
        .load(load),
        .data_in(data_in),
        .next_count(next_count),
        .current_count(count_out)
    );

endmodule

module shift_logic (
    input wire shift_dir,
    input wire [3:0] current_count,
    output reg [3:0] next_count
);

    always @(*) begin
        if (shift_dir) begin
            next_count = {current_count[0], current_count[3:1]};  // Shift right
        end else begin
            next_count = {current_count[2:0], current_count[3]};  // 

In [6]:
new_conv = conv[:30]
new_graph = graph[:30]

In [7]:
for i in range(30):
    new_conv[i]['task_id'] = i

In [8]:
new_conv_path = '/data/LPJ/ICML25/GraphCoder/graphgpt_dataset/gpt_dataset_construction/less30_acl25_gpt4/with_module_head/eval/conversations_30.json'
new_graph_path = '/data/LPJ/ICML25/GraphCoder/graphgpt_dataset/gpt_dataset_construction/less30_acl25_gpt4/with_module_head/eval/graph_30.jsonl'

In [9]:
with open(new_conv_path, 'w') as f:
    json.dump(new_conv, f)

new_graph.to_json(new_graph_path, orient='records', lines=True)

In [10]:
with open(new_conv_path, 'r') as f:
    conv = json.load(f)



In [11]:
conv[0]

{'conversations': [{'from': 'human',
   'value': "Given a submodules interconnection graph: \n<graph>\n, \nnodes: [{'id': 0, 'content': 'CLK_in', 'type': 'input port'}, {'id': 1, 'content': 'RST', 'type': 'input port'}, {'id': 2, 'content': 'CLK_15', 'type': 'output port'}, {'id': 3, 'content': 'CLK_3', 'type': 'output port'}, {'id': 4, 'content': 'div_15', 'type': 'submodule'}, {'id': 5, 'content': 'div_3', 'type': 'submodule'}], \nedge_attrs: [], \nconnectivity: [[0, 1, 0, 1, 4, 5], [4, 4, 5, 5, 2, 3]]\n, Design a hierarchical Verilog module for a frequency divider that accepts a 60MHz input clock and outputs two different clock signals: 15MHz and 3MHz.\n\nModule name:\n    multi_freq_div\n\nInput ports:\n    CLK_in: Input clock signal\n    RST: Reset signal\n\nOutput ports:\n    CLK_15: Output clock signal with a frequency of CLK_in divided by 4.\n    CLK_3: Output clock signal with a frequency of CLK_in divided by 20.\n\nImplementation:\nThe module uses two submodules:\n- clk_div_4